In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa

In [2]:
df = pd.read_csv(r"..\data\raw_german_credit_data.csv")

In [3]:
df.sample(10)

,Duration,Checking account,Credit amount,Purpose,Job,Sex,data_source,Saving accounts,Housing,Risk,Age,Unnamed: 0
4136,24.0,NaN,7814,car,3.0,male,kaggle,little,own,good,38,880.0
1437,15.0,little,975,furniture/equipment,2.0,male,kaggle,little,own,good,25,690.0
3252,48.0,moderate,8358,car,2.0,female,kaggle,quite rich,own,good,30,NaN
1496,9.0,rich,745,radio/TV,1.0,female,kaggle,little,own,bad,28,936.0
2802,9.0,moderate,1549,car,0.0,male,kaggle,NaN,own,good,35,799.0
3627,36.0,moderate,3804,radio/TV,2.0,female,kaggle,little,own,bad,42,301.0
4534,36.0,NaN,7855,car,2.0,female,kaggle,little,own,bad,25,113.0
1052,30.0,rich,1908,business,3.0,male,kaggle,little,own,bad,66,213.0
2082,12.0,little,2579,car,1.0,male,kaggle,little,own,bad,33,416.0
2347,21.0,little,3763,car,1.0,male,kaggle,NaN,own,good,24,906.0


In [4]:
nans = ['NA', 'N/A', 'null', 'NULL', 'nan', 'NaN', '', ' ', '?']
for col in df.columns:
    if len(df[df[col].isin(nans)]) > 0:
        print(f"Weird Nan representation found for column {col}")

No weird representation for Nan values found for columns

In [5]:
for col in df.columns:
    assert len(df[pd.isna(df[col])]) + len(df[df[col].notna()]) == len(df[col]), f"Nan + not nan assertion invalid for column {col}"

We can see that indeed all nan + non nan values match up to the column length.

I'll get rid of Checking account column, because it contains too much Nans in it.

Now let's get rid off data_source column, due to it only contains the value kaggle in it. 

In [6]:
df.drop(columns=["data_source"])

,Duration,Checking account,Credit amount,Purpose,Job,Sex,Saving accounts,Housing,Risk,Age,Unnamed: 0
0,24.0,little,3632,car,2.0,female,little,rent,good,22,613.0
1,24.0,NaN,2603,car,2.0,female,rich,rent,good,28,733.0
2,12.0,little,902,education,2.0,male,little,rent,bad,21,203.0
3,9.0,little,654,car,1.0,male,little,own,bad,28,472.0
4,21.0,little,3414,education,2.0,male,little,own,bad,26,174.0
...,...,...,...,...,...,...,...,...,...,...,...
4841,11.0,moderate,4771,radio/TV,2.0,male,little,own,good,51,77.0
4842,20.0,NaN,3485,car,2.0,male,NaN,own,good,44,901.0
4843,24.0,moderate,3069,furniture/equipment,2.0,male,moderate,free,good,30,NaN
4844,12.0,little,1200,education,2.0,female,NaN,rent,good,23,221.0


Now, let's fix the dtypes
|Column | Description | Type |
|-|-|-|
| Duration | Credit's estimated duration in Months | Numerical continuous |
| Credit amount | Required credit ammount | Numerical continuous |
| Purpose | What is the credit for | Categorical nominal |
| Job | 0 = unskilled and non-resident, 1 = unskilled and resident, 2 = skilled, 3 = highly skilled | Categorical ordinal |
| Sex | | Categorical nominal |
| Saving accounts | Level of savings of the applicant (e.g., little, quite rich, rich) | Categorical ordinal |
| Housing | Type of housing situation (e.g., rent, own, free) | Categorical nominal |
| Risk | bad = 0, 1 = good | Categorical binary |
| Age | Age of the credit requester | Numerical continuous |


First, let's handle categorical types

In [7]:
categorical_cols = {
    "Purpose": [], 
    "Job": [0, 1, 2, 3], 
    "Sex": [],
    "Saving accounts": ['little', 'quite rich', 'rich'], 
    "Housing": [],
    "Risk": ["good", "bad"] 
}

df[list(categorical_cols.keys())] = df[list(categorical_cols.keys())].astype("category")

for col, categories in categorical_cols.items():
    if len(categories) > 0:
        df[col] = pd.Categorical(df[col], categories=categories, ordered=True)

Now, let's handle numerical columns

In [8]:
numeric_cols = {
    "Duration" : "Int64",  # has to be base 64 so it can be nullable
    "Credit amount": "Int64",
    "Age": "Int64"
}

checking_df = df.copy()
for col in numeric_cols.keys():
    checking_df[col] = checking_df[col].astype(str)  # convert to string temporarily for checking
    mask = ~checking_df[col].str.match(r'^-?\d+\.?\d*$', na=True)  # detecting weird values inside numerical columns
    print(f"Invalid values found for column {col}:", df[mask][col].unique())
    df.loc[mask, col] = np.nan

df = df.astype({col: num_type for col, num_type in numeric_cols.items()})

Invalid values found for column Duration: [nan]
Invalid values found for column Credit amount: [nan 'dfas' 'qwretryet' 'ttqweyuet']
Invalid values found for column Age: [nan 'hgd']


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Duration          4699 non-null   Int64   
 1   Checking account  2732 non-null   object  
 2   Credit amount     4568 non-null   Int64   
 3   Purpose           4686 non-null   category
 4   Job               4635 non-null   category
 5   Sex               4617 non-null   category
 6   data_source       4844 non-null   object  
 7   Saving accounts   3306 non-null   category
 8   Housing           4622 non-null   category
 9   Risk              4705 non-null   category
 10  Age               4703 non-null   Int64   
 11  Unnamed: 0        4685 non-null   float64 
dtypes: Int64(3), category(6), float64(1), object(2)
memory usage: 271.1+ KB


In [10]:
schema = pa.Table.from_pandas(df).schema

In [11]:
df.to_parquet(
    r"..\data\types_fixed_german_credit_data.parquet", 
    index = False,
    schema = schema
)